Checking if the 2mmask function made for the files in storeA works correctly on the files in storeB used for the ml project:
$\newline$

Explanation of the code: 

$\newline$
The transformation is an approach following xroms example on xarrays website:
$\newline$
$Z_0 = \frac{h_c \cdot S + h \cdot C}{h_c + h }$
$\newline$
$z = Z_0 (\zeta + h) + \zeta$
$\newline$
This creates a new datacoordinate Z_rho. 

Explanation: 
The code calculates the midpoint of each s-layer depending on which transformation is found in the V-transform. V-transform == 2 is more regulary used for newer ROMS models according to xroms. 
$\newline$
The calculations creates a variable $Z_0$ which is calculated by the use of:
$\newline$
hc = critical depth for stretching 
$\newline$
$s_rho$ = s-layers 
$\newline$
h = total depth
$\newline$
$Cs_r$ = stretching curve for the rho points
$\newline$
$\zeta$ = free surface  
$\newline$

$Z_0$ explains the depth at a given grid cell using the stretching features including the topography, but it excludes the surface layer.
$z_{rho}$ then includes the surface elevation by including $\zeta$ in the calculations and providing the dataset with a new coordinate defining the depth of the grid cells. 

$\newline$
$\newline$

Finding the 2m values:
1. Take the absolute value of the depths (using z_rho) and then subtracting two, to then be able to pick out the resulting numbers closest to zero. Those closest to zero will be the ones near 2meters depth in the model.
2. The diff.fillna function allows us to have zeros as a value for the landvalues rather than nan, as nan gives errors. Though this is corrected later in the code when plotting with seaborn to ensure the land is visibly different from the rest of the plot.  
3. Using argmin, I am selecting out the index of the number closest to zero, to find which s_layer it belongs to. This is why the dimension being called on is s_rho. 
4. Then selecting the index array defined above of those closest to two meters, I use this as an isel, only selecting out these z_rho values to create a new array. 

In [ ]:
#Importing necessary datasets
import xarray as xr
import matplotlib.pyplot as plt 
import numpy as np 

In [ ]:
#Manually installing seaborn due to problems with the terminal
%pip install seaborn
#pip show seaborn
import seaborn as sns

In [ ]:
#Importing the datasets and selecting different timesteps to reduce dataset. 01.06.2024
file1 = f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2024/norkyst800-20240601.nc'
ds_norkyst_lavvann = xr.open_dataset(file1, engine="netcdf4").sel(time = '2024-06-01T07:00:00.000000000') #Kartverket kl. 07 - 30cm i vannstand
ds_norkyst_høyvann = xr.open_dataset(file1, engine = "netcdf4").sel(time = '2024-06-01T05:00:00.000000000') #Kartverket kl. 01 - 70cm i vannstand

In [ ]:
#check datafile 

"""
Differences between the storeA and storeB files
ocean_time = time 
Vtransform does not exist in the storeB norkystv3_hindcast, has to be removed
I use the equation for model vtransform == 2 because it represents newer models according to xroms, which I am guessing this is 
Otherwhise I think everything should be the same between the datasets
"""
#ds_norkyst_lavvann.time.values

ds_norkyst_lavvann

In [ ]:
def finding_2m_layer(ds_name, title):
    #Define necessary variables used for the transformation from s_layer to depth
    hc = ds_name["hc"] #Critical depth for stretching
    cs_r = ds_name["Cs_r"] #stretching curve at rho points
    zeta = ds_name["zeta"] #.fillna(0) #free-surface 
    H = ds_name["h"] #bathymetry at rho-points (depth)
    #Vtransform = ds_name["Vtransform"] Not in this dataset
    s_rho = ds_name["s_rho"] #range 1,40. 40 is surface layer

    #Transformation process
    #if Vtransform == 1:
        #Z_0_rho = hc * (s_rho - cs_r) + cs_r * H
        #z_rho = Z_0_rho + zeta * (1+Z_0_rho/H)
    #elif Vtransform == 2:
    Z_0_rho = (hc * s_rho + cs_r * H) / (hc + H)
    z_rho = zeta + (zeta + H) * Z_0_rho

    ds_name.coords["z_rho"] = z_rho.transpose() #Corrects the dimensions

    #calculating the 2meters depth and creating two new arrays. One controlrun
    #for the actual values and one array containing the indexes of the values. 
    #If one wants to find for other depths than 2 meters, replace the calculatipon of diff with the new value. 
    z_rho_pos = abs(ds_name["z_rho"])
    diff = abs(z_rho_pos-2) #2meter
    diff = diff.fillna(0)
    index = diff.argmin(dim = "s_rho")
    close_to_2m_val = z_rho.isel(s_rho=index)
    #zero = nan in plot
    index = index.where(index != 0)
    ax = sns.heatmap(np.array(index)[::-1,::-1], vmin=25, cbar_kws={'label': 'Index'})
    ax.set_title(title)
    return index, close_to_2m_val, ax


In [ ]:
finding_2m_layer(ds_norkyst_lavvann, title="Lavvann 01.06.2024 kl.07 - 30cm")

In [ ]:
finding_2m_layer(ds_norkyst_høyvann, title="Høyvann 01.06.2024 kl. 01 - 70cm")

In [ ]:
#Adding a later year to see wether there are many changes or not
#Same date but in 2018 and 2012

In [ ]:
file_2 = f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2018/norkyst800-20180601.nc'
file_3 = f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2012/norkyst800-20120601.nc'

In [ ]:
ds_norkyst_høyvann_2018 = xr.open_dataset(file_2, engine="netcdf4").sel(time = '2018-06-01T20:00:00.000000000')
ds_norkyst_høyvann_2012 = xr.open_dataset(file_3, engine="netcdf4").sel(time = '2012-06-01T15:00:00.000000000' )
ds_norkyst_lavvann_2018 = xr.open_dataset(file_2, engine="netcdf4").sel(time = '2018-06-01T01:00:00.000000000')
ds_norkyst_lavvann_2012 = xr.open_dataset(file_3, engine = "netcdf4").sel(time = '2012-06-01T08:00:00.000000000' )

In [ ]:
ds_norkyst_høyvann_2012.time
#lavvannet og høyvannet var ikke veldig markant denne dagen, men har likevel tatt med de to tidspunktene som skiller seg mest ut

In [ ]:
finding_2m_layer(ds_norkyst_lavvann_2012, title="01.06.2012 lavvann kl. 08 - 37cm")

In [ ]:
finding_2m_layer(ds_norkyst_høyvann_2012, title="01.06.2012 kl. 15 høyvann - 74cm")

In [ ]:
finding_2m_layer(ds_norkyst_høyvann_2018, title="01.06.2018 kl.20 høyvann - 59cm")

In [ ]:
finding_2m_layer(ds_norkyst_lavvann_2018, title="01.06.2018 kl. 01 lavvann - 33cm")

In [ ]:
#Controlcheck -> checking wether the values from the non-transformed dataset corresponds with the values that have been transformed. 
ds_norkyst_lavvann_2018.sel(X = 800, Y = 800 )

In [ ]:
#Seperate the functions, one that transforms and one that plots:
#for time effiency 

def transformation(ds_name):
    #Define necessary variables used for the transformation from s_layer to depth
    hc = ds_name["hc"] #Critical depth for stretching
    cs_r = ds_name["Cs_r"] #stretching curve at rho points
    zeta = ds_name["zeta"] #.fillna(0) #free-surface 
    H = ds_name["h"] #bathymetry at rho-points (depth)
    #Vtransform = ds_name["Vtransform"] Not in this dataset
    s_rho = ds_name["s_rho"] #range 1,40. 40 is surface layer

    #Transformation process
    #if Vtransform == 1:
        #Z_0_rho = hc * (s_rho - cs_r) + cs_r * H
        #z_rho = Z_0_rho + zeta * (1+Z_0_rho/H)
    #elif Vtransform == 2:
    Z_0_rho = (hc * s_rho + cs_r * H) / (hc + H)
    z_rho = zeta + (zeta + H) * Z_0_rho

    ds_name.coords["z_rho"] = z_rho.transpose() #Corrects the dimensions
    return z_rho

def add_coordinate(ds_name):
    ds_name["z_rho"] = transformation(ds_name)
    return ds_name

def plot_the_2m_layer(ds_name, title):
    add_coordinate(ds_name = ds_name)
    transformation(ds_name=ds_name)
    z_rho = ds_name["z_rho"]
    z_rho_pos = abs(ds_name["z_rho"])
    diff = abs(z_rho_pos-2) #2meter
    diff = diff.fillna(0)
    index = diff.argmin(dim = "s_rho")
    close_to_2m_val = z_rho.isel(s_rho=index)
    #zero = nan in plot
    index = index.where(index != 0)
    ax = sns.heatmap(np.array(index)[::-1,::-1], vmin=25, cbar_kws={'label': 'Index'})
    ax.set_title(title)
    return index, close_to_2m_val, ax


In [ ]:
finding_2m_layer(ds_norkyst_lavvann_2018, title = "sjalala")

In [ ]:
#Funker men output har bare kollapset
test = plot_the_2m_layer(ds_norkyst_lavvann_2018,title="forsøk på å dele opp funk")

In [ ]:
test

In [ ]:
file1 = f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2024/norkyst800-20240601.nc'
ds_val = xr.open_dataset(file1, engine="netcdf4").isel(X = 300, Y = 760,time = 0 )
ds = xr.open_dataset(file1, engine="netcdf4").isel(time = 0)

In [ ]:
#ds
ds_val

In [ ]:
lon = ds["lon"]
lat = ds["lat"]
temp = ds["temperature"]

lon_val = ds_val["lon"]
lat_val = ds_val["lat"]
temp_val = ds_val["temperature"]

In [ ]:
#checking if selected gridpoint is in ocean
plt.pcolormesh(lon, lat, temp[0,:,:])
plt.scatter(lon_val,lat_val,temp_val[0], color ="red")

In [ ]:
transformation(ds_val)

In [ ]:
#z_rho adds up for first two functions. Printing out a list to get the depth for all layers
add_coordinate(ds_val)

In [ ]:
#Seperate the functions, one that transforms and one that plots:
#for time effiency 

def transformation(ds_name):
    #Define necessary variables used for the transformation from s_layer to depth
    hc = ds_name["hc"] #Critical depth for stretching
    cs_r = ds_name["Cs_r"] #stretching curve at rho points
    zeta = ds_name["zeta"] #.fillna(0) #free-surface 
    H = ds_name["h"] #bathymetry at rho-points (depth)
    #Vtransform = ds_name["Vtransform"] Not in this dataset
    s_rho = ds_name["s_rho"] #range 1,40. 40 is surface layer

    #Transformation process
    #if Vtransform == 1:
        #Z_0_rho = hc * (s_rho - cs_r) + cs_r * H
        #z_rho = Z_0_rho + zeta * (1+Z_0_rho/H)
    #elif Vtransform == 2:
    Z_0_rho = (hc * s_rho + cs_r * H) / (hc + H)
    z_rho = zeta + (zeta + H) * Z_0_rho

    ds_name.coords["z_rho"] = z_rho.transpose() #Corrects the dimensions
    return z_rho

def add_coordinate(ds_name):
    ds_name["z_rho"] = transformation(ds_name)
    return ds_name

def plot_the_2m_layer_grid(ds_name):
    add_coordinate(ds_name = ds_name)
    transformation(ds_name=ds_name)
    z_rho = ds_name["z_rho"]
    z_rho_pos = abs(ds_name["z_rho"])
    diff = abs(z_rho_pos-2) #2meter
    diff = diff.fillna(0)
    index = diff.argmin(dim = "s_rho")
    close_to_2m_val = z_rho.isel(s_rho=index)
    #zero = nan in plot
    index = index.where(index != 0)
    #ax = sns.heatmap(np.array(index), vmin=25, cbar_kws={'label': 'Index'})
    #ax.set_title(title)
    return f'this is the selected {index}', close_to_2m_val

In [ ]:
plot_the_2m_layer_grid(ds_val)

In [ ]:
#It picks out layer 37 which is the value nearest two meters. it seems to be correct